In [251]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import datetime as dt
import numpy as np

In [252]:
df = pd.read_csv("../data/orders.csv")

In [253]:
orders = df.T.reset_index().T.reset_index(drop=True)
orders.columns = ["order_id", "activation_time_local", "country_code", "store_address", "final_status", 
              "payment_status", "products", "products_total", "purchase_total_price"]

orders = orders[orders.purchase_total_price != 0]
orders.reset_index(drop=True, inplace=True)

In [254]:
for i in orders:
    try:
        orders[i] = orders[i].apply(lambda x: pd.Timestamp(x))
        continue
    except:
        pass
    try:
        orders[i] = orders[i].astype("float")
        continue
    except:   
        pass    
    
orders.dtypes

order_id                        float64
activation_time_local    datetime64[ns]
country_code                     object
store_address                   float64
final_status                     object
payment_status                   object
products                        float64
products_total                  float64
purchase_total_price            float64
dtype: object

In [255]:
orders["under_authorized"] = orders.products_total < orders.purchase_total_price

In [256]:
orders["under_authorized_w20"] = ((orders.purchase_total_price - orders.products_total) / orders.products_total) < 0.2
orders["under_authorized_w20"].mean() * 100

65.56798116539142

In [257]:
orders.groupby(["country_code"])["under_authorized"].mean() * 100

country_code
AR     70.475183
BR    100.000000
CI    100.000000
CL    100.000000
CR     48.112294
DO     68.831169
EC     70.906690
EG     70.508890
ES     67.175508
FR     69.960080
GE     59.453782
GT     36.434109
IT     61.238170
KE     62.352941
MA     47.192608
PA     67.248908
PE     60.370120
PR     48.275862
PT     68.482490
RO     50.552764
TR     55.419611
UA     54.164413
UY     56.250000
Name: under_authorized, dtype: float64

In [258]:
def remainder_amount(x,increment=0.2):
    return (1 + increment) * x
        
orders["remainder_amount"] = [np.NaN if orders.under_authorized_w20[i] else remainder_amount(orders["products_total"][i]) for i in range(len(orders))]
orders


,order_id,activation_time_local,country_code,store_address,final_status,payment_status,products,products_total,purchase_total_price,under_authorized,under_authorized_w20,remainder_amount
0,33557880.0,2019-03-10 23:59:59,AR,14200.0,DeliveredStatus,PAID,1.0,4.54,8.64,True,False,5.448
1,33512615.0,2019-03-10 23:58:32,TR,28725.0,DeliveredStatus,PAID,1.0,3.76,3.76,False,True,NaN
2,33512451.0,2019-03-10 23:57:56,TR,28725.0,DeliveredStatus,PAID,1.0,2.86,2.86,False,True,NaN
3,33557765.0,2019-03-10 23:57:21,AR,34565.0,DeliveredStatus,PAID,2.0,2.86,6.48,True,False,3.432
4,33512273.0,2019-03-10 23:57:13,TR,63536.0,DeliveredStatus,PAID,4.0,1.88,1.96,True,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
54363,31961779.0,2019-03-01 00:09:57,TR,34953.0,DeliveredStatus,PAID,1.0,1.80,1.80,False,True,NaN
54364,32002340.0,2019-03-01 00:09:21,AR,14455.0,DeliveredStatus,NOT_PAID,2.0,5.14,5.14,False,True,NaN
54365,31960607.0,2019-03-01 00:04:31,TR,68820.0,DeliveredStatus,PAID,2.0,10.17,10.33,True,True,NaN
54366,32002046.0,2019-03-01 00:03:06,AR,55159.0,DeliveredStatus,PAID,1.0,4.24,4.91,True,True,NaN


In [259]:
problematic_stores = orders.groupby("store_address")["remainder_amount"].mean().reset_index()
problematic_stores = problematic_stores.sort_values("remainder_amount", ascending=False)
problematic_stores.head(10)

,store_address,remainder_amount
506,10151.0,126.720
417,8863.0,103.200
698,13477.0,100.800
2154,46310.0,93.480
1081,19247.0,79.200
4853,70743.0,75.912
204,3304.0,58.980
717,13812.0,57.600
1793,36937.0,57.396
944,16769.0,55.920


In [281]:
# For under-auth orders is there a correlation between the difference in the prices
# and the cancellation of the order? In other words: Is an order more likely to be
# cancelled as the price difference increases? 

canceled_str = orders.final_status.unique()[1]

def canceled(x, string=canceled_str):
    if x == string:
        return True
    else: 
        return False
    
orders["price_diff"] = orders.purchase_total_price - orders.products_total
#orders["canceled"] = orders["final_status"].apply(lambda x: canceled(x))


orders_under = orders[orders.under_authorized]
(orders.purchase_total_price == 0).sum()


pd.concat([orders.groupby("final_status")["price_diff"].mean(), 
           orders_under.groupby("final_status")["price_diff"].mean()],axis=1)



,price_diff,price_diff
final_status,,
CanceledStatus,7.835177,12.447249
DeliveredStatus,2.055517,3.636929


In [278]:
x = orders.groupby(["final_status","under_authorized"])["price_diff"].mean().reset_index()
y = x.pivot(index="under_authorized", columns = ["final_status"])
y.index

Index([False, True], dtype='bool', name='under_authorized')

,final_status,under_authorized,price_diff
0,CanceledStatus,False,-1.334706
1,CanceledStatus,True,12.447249
2,DeliveredStatus,False,-0.749685
3,DeliveredStatus,True,3.636929


In [280]:
orders.groupby(["final_status","under_authorized"])["price_diff"].mean()

final_status     under_authorized
CanceledStatus   False               -1.334706
                 True                12.447249
DeliveredStatus  False               -0.749685
                 True                 3.636929
Name: price_diff, dtype: float64